# Entraînement et Optimisation des Hyperparamètres d'un Modèle de Machine Learning dans le Cloud

Dans ce notebook, nous allons vous montrer comment entraîner un modèle d'apprentissage automatique sur le cloud et optimiser les hyperparamètres. Cette approche vous permettra d'éviter les plantages du kernel et les longs temps d'exécution.

In [ ]:
from azure.ai.ml.entities import Environment
from azure.ai.ml import MLClient, Input, Output
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import UserIdentityConfiguration
from azure.ai.ml import command
from azure.ai.ml.entities import AmlCompute
import os

#### **Workspace**

Tout d'abord, vous devrez vous connecter à votre espace de travail Azure ML. L'espace de travail est la ressource de niveau supérieur pour Azure Machine Learning, fournissant un emplacement centralisé pour travailler avec tous les éléments que vous créez lorsque vous utilisez Azure Machine Learning.

Nous utilisons DefaultAzureCredential pour accéder à l'espace de travail. DefaultAzureCredential devrait être capable de gérer la plupart des scénarios d'authentification du kit SDK Azure."

Dans la cellule suivante, saisissez votre ID d'abonnement, le nom du groupe de ressources et le nom de l'espace de travail. Pour trouver l'ID d'abonnement et le nom du groupe de ressources :

- Dans la barre d'outils de Azure Machine Learning Studio en haut à droite, sélectionnez le nom de votre espace de travail.
- Copiez la valeur du groupe de ressources et de l'ID d'abonnement dans le code.

In [2]:
subscription_id = "votre subscription_id"
resource_group = "votre resource_group"
workspace_name = "nom"
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace_name
)

#### **Creating my Environment**

Pour exécuter une tâche AzureML, vous aurez besoin d'un environnement. Un environnement est le runtime logiciel et les bibliothèques que vous souhaitez installer sur la machine de calcul sur laquelle vous effectuerez la formation. Il est similaire à votre environnement Python sur votre machine locale.

AzureML propose de nombreux environnements préconfigurés ou prêts à l'emploi qui sont utiles pour des scénarios de formation et d'inférence courants. Vous pouvez également créer vos propres environnements "personnalisés" en utilisant une image Docker ou une configuration conda.

In [3]:
custom_env_name = "sklearn_envs"
job_env = Environment(
    name=custom_env_name,
    #chemin vers le fichier .yaml dans votre répertoire
    description="Custom environment for sklearn - classification",
    conda_file="././dependencies.yaml",
    image="choose the image:latest",
)
job_env = ml_client.environments.create_or_update(job_env)

print(
    f"Environment with name {job_env.name} is registered to workspace, the environment version is {job_env.version}"
)

Environment with name sklearn_envs is registered to workspace, the environment version is 91


AzureML a besoin d'une ressource de calcul pour exécuter une tâche. Il peut s'agir de machines mono-noeud ou multi-noeuds (cluster) avec un système d'exploitation Linux ou Windows, ou d'une infrastructure de calcul spécifique comme Spark.

Pour cet exemple, nous avons besoin d'un cluster basique. Optons pour un modèle Standard_DS3_v2 avec 2 cœurs vCPU, 7 Go de RAM et créons une ressource de calcul Azure ML.

Si le cluster ou la machine virtuelle est déjà présente, il suffit de spécifier son nom.

In [ ]:
cpu_cluster = AmlCompute(
        name='instance',
        type="amlcompute",
        # Virtual Machine family
        size="STANDARD_DS3_V2",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=4,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=180,
    )

    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster).result()

print(
    f"AMLCompute with name {cpu_cluster.name} is created, the compute size is {cpu_cluster.size}"
)

#### **Launching the job to train the model**

Maintenant que vous avez tous les éléments nécessaires pour exécuter votre tâche, il est temps de créer la tâche elle-même à l'aide du kit SDK Python Azure ML v2. Nous allons créer une tâche de type "commande".

Une tâche de type "commande" AzureML est une ressource qui spécifie tous les détails nécessaires pour exécuter votre code de formation dans le cloud : les entrées et les sorties, le type de matériel à utiliser, les logiciels à installer et la manière d'exécuter votre code. La tâche de type "commande" contient les informations pour exécuter une seule commande.

Le job exécutera le script de formation "Decision Tree.py" en Python.

In [6]:
python_job = command(
    code="./src", #chemin vers le script
    display_name="name of the experiment",
    environment= job_env,
    compute= "instance",
    
    inputs={
        "input_data": Input(
            type="uri_folder",
            path="path of the input data",
        ),
        "criterion" : "entropy",
        "max_depth" : 50,
        "min_samples_leaf" :20,
        "min_samples_split" : 1000,
        "min_impurity_decrease" : 0.0001,    
        "random_state" : 42,
        #"ccp_alpha" : 0.2,
        },

    outputs={
        "model_output": Output(
            type="uri_folder",
            path="path to save the model",
        ),
    },
    command="python Decision Tree.py \
    --input_data ${{inputs.input_data}} \
    --criterion ${{inputs.criterion}} --max_depth ${{inputs.max_depth}}\
    --min_samples_leaf ${{inputs.min_samples_leaf}} --min_samples_split ${{inputs.min_samples_split}}\
    --min_impurity_decrease ${{inputs.min_impurity_decrease}} --random_state ${{inputs.random_state}}\
     --model_output ${{outputs.model_output}}"
)

returned_job = ml_client.jobs.create_or_update(python_job)

ml_client.jobs.stream(returned_job.name)

Uploading src (0.18 MBs): 100%|██████████| 180203/180203 [00:00<00:00, 363538.41it/s]




RunId: dreamy_wire_9qv3m082cm
Web View: https://ml.azure.com/runs/dreamy_wire_9qv3m082cm?wsid=/subscriptions/82ad5388-182f-442b-95e2-3378756be384/resourcegroups/rg-e7-e2-sbx-app-dlf-bas-01/workspaces/aml-e7-e2-fr-sbx-dlf-bas-001


#### **Hyperparameter tuning**

Le "sweep job" est un processus essentiel dans le domaine de l'apprentissage automatique visant à trouver les paramètres optimaux pour un modèle. Les hyperparamètres, qui sont des réglages qui ne sont pas appris par le modèle lui-même, jouent un rôle crucial dans la performance d'un algorithme d'apprentissage automatique.

Une tâche de recherche d'hyperparamètres consiste à parcourir différentes combinaisons d'hyperparamètres pour déterminer celles qui produisent les meilleurs résultats en termes de précision, de généralisation et d'autres mesures de performance. Cette approche permet d'optimiser la configuration du modèle sans avoir à effectuer manuellement une multitude d'expérimentations fastidieuses.

L'optimisation des hyperparamètres dans le cloud présente des avantages considérables. L'utilisant des ressources infiniment évolutives du cloud peut accélérer les expérimentations, explorer des espaces de recherche plus vastes, réaliser des économies de coûts, et assurer la sécurité et la sauvegarde des données.

In [9]:
from azure.ai.ml.sweep import Choice

job_for_sweep = python_job(
    max_depth=Choice(values=[50, 150, 500, 1000]),
    min_samples_split=Choice(values=[10, 50, 100, 300, 500,1000]),
    min_impurity_decrease=Choice(values=[0.0001, 0.001, 0.01, 0.1]),
    min_samples_leaf=Choice(values=[20, 50, 100, 300, 500, 800]),
    criterion=Choice(values=['gini', 'entropy']),
    ccp_alpha=Choice(values=[0.0, 0.001, 0.01, 0.1, 0.3, 0.6]),
)

In [11]:
sweep_job = job_for_sweep.sweep(
    sampling_algorithm="grid", # we can use "random"
    primary_metric="f1_score",
    goal="Maximize",
    #max_total_trials=200,
    #max_concurrent_trials=500, 
)
returned_sweep_job = ml_client.create_or_update(sweep_job)

ml_client.jobs.stream(returned_sweep_job.name)
returned_sweep_job = ml_client.jobs.get(name=returned_sweep_job.name)

RunId: nifty_quince_zzvk0qrqj0
Web View: https://ml.azure.com/runs/nifty_quince_zzvk0qrqj0?wsid=/subscriptions/82ad5388-182f-442b-95e2-3378756be384/resourcegroups/rg-e7-e2-sbx-app-dlf-bas-01/workspaces/aml-e7-e2-fr-sbx-dlf-bas-001

Streaming azureml-logs/hyperdrive.txt

[2023-08-18T11:56:20.592647][GENERATOR][INFO]Trying to sample '1000' jobs from the hyperparameter space
[2023-08-18T11:56:27.8656175Z][SCHEDULER][INFO]Scheduling job, id='nifty_quince_zzvk0qrqj0_1' 
[2023-08-18T11:56:27.8645210Z][SCHEDULER][INFO]Scheduling job, id='nifty_quince_zzvk0qrqj0_0' 
[2023-08-18T11:56:28.0030341Z][SCHEDULER][INFO]Scheduling job, id='nifty_quince_zzvk0qrqj0_2' 
[2023-08-18T11:56:28.1068804Z][SCHEDULER][INFO]Scheduling job, id='nifty_quince_zzvk0qrqj0_3' 
[2023-08-18T11:56:28.1576955Z][SCHEDULER][INFO]Successfully scheduled a job. Id='nifty_quince_zzvk0qrqj0_0' 
[2023-08-18T11:56:28.1865160Z][SCHEDULER][INFO]Successfully scheduled a job. Id='nifty_quince_zzvk0qrqj0_1' 
[2023-08-18T11:56:28.219653

In [31]:
best_run = returned_sweep_job.properties["best_child_run_id"]
best_run

'jovial_hominy_r8my72krcx_90'

In [32]:
returned_sweep_job.properties["score"]

'11.822784810126583'

In [33]:
returned_sweep_job.properties["best_metric_status"]

'Succeeded'